# Face Data Collection
Install required packages and import libraries

In [ ]:
!pip install mtcnn opencv-python
import cv2
from mtcnn import MTCNN
from google.colab import files
from IPython.display import display, Javascript
from google.colab.patches import cv2_imshow
import numpy as np
import os

In [ ]:
def capture_faces():
    from IPython.display import HTML, Image
    from google.colab.output import eval_js
    from base64 import b64decode
    import uuid

    SAVE_DIR = 'face_data'
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

    person_name = input("Enter person's name: ")
    person_dir = os.path.join(SAVE_DIR, person_name)
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)

    detector = MTCNN()

    def take_photo(filename='photo.jpg'):
        js = Javascript('''
            async function takePhoto() {
                const stream = await navigator.mediaDevices.getUserMedia({video: true});
                const video = document.createElement('video');
                const div = document.createElement('div');
                document.body.appendChild(div);
                div.appendChild(video);
                video.srcObject = stream;
                await video.play();

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                const img = canvas.toDataURL('image/jpeg');

                stream.getVideoTracks()[0].stop();
                div.remove();
                return img;
            }
            ''')
        display(js)
        data = eval_js('takePhoto()')
        binary = b64decode(data.split(',')[1])
        return binary

    num_photos = 20  # Number of photos to capture
    for i in range(num_photos):
        print(f"Taking photo {i+1}/{num_photos}")
        binary = take_photo()
        img_file = f"{person_dir}/{str(uuid.uuid4())}.jpg"
        
        # Save the image
        with open(img_file, 'wb') as f:
            f.write(binary)
        
        # Read and process the saved image
        img = cv2.imread(img_file)
        faces = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        if faces:
            for face in faces:
                x, y, width, height = face['box']
                face_img = img[y:y+height, x:x+width]
                face_img = cv2.resize(face_img, (160, 160))
                cv2.imwrite(img_file, face_img)
        else:
            print("No face detected in this photo")
            os.remove(img_file)

    print("Data collection completed!")

In [ ]:
# Run the face collection process
capture_faces()